In [1]:
import requests
import pandas as pd
from owslib.wfs import WebFeatureService
import os

pyproj not installed


In [2]:
WFS_URL = "https://gdi.berlin.de/services/wfs/baumbestand"
LAYERS = ["baumbestand:strassenbaeume", "baumbestand:anlagenbaeume"]
BATCH_SIZE = 10000
OUTPUT_FILENAME = "baumbestand_berlin.csv"
OUTPUT_FOLDER = "../data"

In [3]:
wfs = WebFeatureService(url=WFS_URL, version="2.0.0")

In [4]:
wfs.identification.title

'Baumbestand Berlin'

In [5]:
for key in list(wfs.contents):
    print(key)

feature_type = list(wfs.contents)[0]
schema = wfs.get_schema(feature_type)

print(schema)

baumbestand:anlagenbaeume
baumbestand:strassenbaeume
{'properties': {'gisid': 'string', 'pitid': 'string', 'standortnr': 'string', 'kennzeich': 'string', 'namenr': 'string', 'art_dtsch': 'string', 'art_bot': 'string', 'gattung_deutsch': 'string', 'gattung': 'string', 'art_gruppe': 'string', 'pflanzjahr': 'string', 'standalter': 'double', 'kronedurch': 'double', 'stammumfg': 'int', 'baumhoehe': 'double', 'eigentuemer': 'string', 'bezirk': 'string'}, 'required': ['gisid'], 'geometry': 'Point', 'geometry_column': 'geom'}


In [6]:
all_data_frames = []

print("Starte Download aller Berliner Bäume...")

for layer in LAYERS:
    start_index = 0
    print(f"\n--- Starte Layer: {layer} ---")
    
    while True:

        params = {
            "service": "WFS",
            "version": "2.0.0",
            "request": "GetFeature",
            "typeNames": layer,
            "startIndex": start_index,
            "count": BATCH_SIZE,
            "outputFormat": "application/json"
        }
        
        try:
            response = requests.get(WFS_URL, params=params)
            response.raise_for_status()
            
            data = response.json()
            features = data.get("features", [])
            
            df_batch = pd.json_normalize([f['properties'] for f in features])
            
            # 2. Geodaten extrahieren (Koordinaten)
            # GeoJSON Struktur: feature -> geometry -> coordinates [lon, lat]
            coordinates = []
            for f in features:
                geo = f.get('geometry')
                if geo and 'coordinates' in geo:
                    coordinates.append(geo['coordinates'])
                else:
                    coordinates.append([None, None]) # Leere Werte falls Geometrie fehlt
            
            df_batch['longitude'] = [c[0] for c in coordinates]
            df_batch['latitude'] = [c[1] for c in coordinates]
            
            df_batch['source_layer'] = layer
            
            all_data_frames.append(df_batch)
            
            print(f"  Habe {len(features)} Bäume geladen (ab Index {start_index})...")
            
            start_index += BATCH_SIZE
            
            if len(features) < BATCH_SIZE:
                print(f"  Ende des Layers {layer} erreicht.")
                break
                
        except Exception as e:
            print(f"  FEHLER beim Laden ab Index {start_index}: {e}")
            break

Starte Download aller Berliner Bäume...

--- Starte Layer: baumbestand:strassenbaeume ---
  Habe 10000 Bäume geladen (ab Index 0)...
  Habe 10000 Bäume geladen (ab Index 10000)...
  Habe 10000 Bäume geladen (ab Index 20000)...
  Habe 10000 Bäume geladen (ab Index 30000)...
  Habe 10000 Bäume geladen (ab Index 40000)...
  Habe 10000 Bäume geladen (ab Index 50000)...
  Habe 10000 Bäume geladen (ab Index 60000)...
  Habe 10000 Bäume geladen (ab Index 70000)...
  Habe 10000 Bäume geladen (ab Index 80000)...
  Habe 10000 Bäume geladen (ab Index 90000)...
  Habe 10000 Bäume geladen (ab Index 100000)...
  Habe 10000 Bäume geladen (ab Index 110000)...
  Habe 10000 Bäume geladen (ab Index 120000)...
  Habe 10000 Bäume geladen (ab Index 130000)...
  Habe 10000 Bäume geladen (ab Index 140000)...
  Habe 10000 Bäume geladen (ab Index 150000)...
  Habe 10000 Bäume geladen (ab Index 160000)...
  Habe 10000 Bäume geladen (ab Index 170000)...
  Habe 10000 Bäume geladen (ab Index 180000)...
  Habe 10000

In [7]:
if all_data_frames:
    full_df = pd.concat(all_data_frames, ignore_index=True)
    
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"Ordner '{OUTPUT_FOLDER}' wurde neu erstellt.")
    
    full_output_path = os.path.join(OUTPUT_FOLDER, OUTPUT_FILENAME)
    
    full_df.to_parquet(full_output_path, index=False)
    
    print(f"{len(full_df)} Bäume gespeichert.")
    print(f"Datei liegt hier: {full_output_path}")
    print("-" * 30)
    
    print("Vorschau:")
    print(full_df.head())

else:
    print("Es wurden keine Daten gefunden/geladen.")

Ordner '../data' wurde neu erstellt.


/var/folders/0c/r0htcd1j05v_lb53s4xtzbn80000gn/T/ipykernel_40849/2455207046.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(all_data_frames, ignore_index=True)


945907 Bäume gespeichert.
Datei liegt hier: ../data/baumbestand_berlin.csv
------------------------------
Vorschau:
               gisid              pitid standortnr kennzeich  \
0  00008100_000bbafb  00008100:000bbafb         93     01414   
1  00008100_000bbafd  00008100:000bbafd         91     01414   
2  00008100_000bbafe  00008100:000bbafe         90     01414   
3  00008100_000bbaff  00008100:000bbaff         89     01414   
4  00008100_000bbb00  00008100:000bbb00         88     01414   

               namenr                art_dtsch                        art_bot  \
0  Fritz-Reuter-Allee      Pyramiden-Hainbuche  Carpinus betulus 'Fastigiata'   
1  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn            Acer pseudoplatanus   
2  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn            Acer pseudoplatanus   
3  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn            Acer pseudoplatanus   
4  Fritz-Reuter-Allee  Berg-Ahorn, Weiss-Ahorn            Acer pseudoplatanus   

  gattung_de